# Get OSMnx Extraction

In [13]:
import geopandas as gpd
import pandas as pd
import osmnx as ox

## Remote I/O

In [16]:
boundary_filename = "nw_pdx.geojson"
osm_filename = "oregon-latest.osm"

In [17]:
nw_pdx_gdf = gpd.read_file(boundary_filename)

## OSMnx Extraction

In [10]:
# Option 1: use OSMnx to fetch
graph = ox.graph_from_polygon(
    nw_pdx_gdf.to_crs(4326).geometry.unary_union, 
    network_type = "drive", 
    simplify = True
)

In [ ]:
# Option 2: read from disk
graph = ox.graph_from_xml(osm_filename)

In [ ]:
small_graph = ox.truncate_graph_bbox(
    graph, 
    nw_pdx_gdf.bounds.maxy[0], 
    nw_pdx_gdf.bounds.miny[0], 
    nw_pdx_gdf.bounds.maxx[0], 
    nw_pdx_gdf.bounds.minx[0]
)

In [ ]:
link_gdf = ox.graph_to_gdfs(small_graph, nodes = False, edges = True)

In [ ]:
link_gdf['osmid'] = (
 pd.DataFrame(link_gdf.osmid.tolist())
   .fillna('')
   .astype(str)
   .agg(','.join, 1)
   .str.strip(',')
)

link_gdf.head()

In [ ]:
osmnx_gdf = link_gdf[['osmid', 'u', 'v', 'geometry']]
osmnx_gdf.to_file('osmnx_link_disk.geojson', driver='GeoJSON')